In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-ml-w4p2/train.csv
/kaggle/input/2024-ml-w4p2/test.csv
/kaggle/input/2024-ml-w4p2/sample_submit.csv


In [24]:
#랜덤시드 고정
import random
import os

seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [25]:
#pandas 라이브러리를 통한 데이터 불러오기

train = pd.read_csv('/kaggle/input/2024-ml-w4p2/train.csv')
test = pd.read_csv('/kaggle/input/2024-ml-w4p2/test.csv')
submit = pd.read_csv('/kaggle/input/2024-ml-w4p2/sample_submit.csv')

In [26]:
#데이터 실수화
from sklearn.preprocessing import LabelEncoder
classle = LabelEncoder()

train['model'] = classle.fit_transform(train['model'].values)
test['model'] = classle.fit_transform(test['model'].values)
print(train['model'])

0        15
1        87
2        26
3        26
4         2
         ..
34165     0
34166    25
34167    25
34168    33
34169    25
Name: model, Length: 34170, dtype: int64


In [27]:
#데이터 분할
from sklearn.model_selection import train_test_split

X = train.drop('price', axis = 1)
y = train['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

In [35]:
#정규화
from sklearn.preprocessing import StandardScaler

std=StandardScaler()

X_train_std = std.fit_transform(X_train)
X_test_std = std.fit_transform(X_test)
test_std = std.fit_transform(test)

In [36]:
#KNN 회귀
from sklearn.neighbors import KNeighborsRegressor
regressor = KNeighborsRegressor(n_neighbors = 5, p = 1, weights = "distance")

regressor.fit(X_train_std, y_train)

KNeighborsRegressor(p=1, weights='distance')

In [44]:
#성능 측정
print('{:.4f}'.format(regressor.score(X_test_std, y_test)))

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# 예측
predictions = regressor.predict(X_test_std)

# 성능 평가
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print(f"MAE: {mae}, RMSE: {rmse}, R^2: {r2}")

0.9318
MAE: 1556.0666326738965, RMSE: 2504.337877305991, R^2: 0.9318449578629846


In [45]:
#test 예측
test_pred = regressor.predict(test_std)

In [46]:
#submit 저장
submit['price'] = test_pred
print(submit.head())
submit.to_csv('submit.csv', mode='w', header= True, index= False)

   ID         price
0   0   7823.146728
1   1   9343.984231
2   2  18603.120134
3   3  13573.774248
4   4  10721.845672
